In [32]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import chisquare
from scipy.stats import fisher_exact
from utils import load_env_file, set_mpl_configs
from utils import leave_percentile, distribution_analysis

load_env_file()
set_mpl_configs()

DATA_DIR = os.getenv('DATA_DIR')
print('DATA_DIR: {}'.format(DATA_DIR))

load env file
  root dir:
    /Users/k/Repo/gp-ibd
  current system:
    Darwin
  load .env.darwin
  loaded data dir:
    /Users/k/Nutstore Files/毕设-EHR/DB
done.
set matplotlib configs
  font family:
    ['Times New Roman']
done.
DATA_DIR: /Users/k/Nutstore Files/毕设-EHR/DB


In [15]:
df_ibd = pd.read_csv(os.path.join(DATA_DIR, 'complication', 'Complications_Patients.csv'))
df_no_ibd = pd.read_csv(os.path.join(DATA_DIR, 'complication', 'Complications_Patients_NoIBD.csv'))
df_no_ibd.head(100)

,subject_id,hadm_id,seq_num,icd_code,icd_version,subject_id.1,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,22595853,1,5723,9,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000032,22595853,2,78959,9,10000032,F,52,2180,2014 - 2016,2180-09-09
2,10000032,22595853,3,5715,9,10000032,F,52,2180,2014 - 2016,2180-09-09
3,10000032,22595853,4,07070,9,10000032,F,52,2180,2014 - 2016,2180-09-09
4,10000032,22595853,5,496,9,10000032,F,52,2180,2014 - 2016,2180-09-09
...,...,...,...,...,...,...,...,...,...,...,...
95,10000764,27897940,2,41071,9,10000764,M,86,2132,2014 - 2016,NaN
96,10000764,27897940,3,5849,9,10000764,M,86,2132,2014 - 2016,NaN
97,10000764,27897940,4,2875,9,10000764,M,86,2132,2014 - 2016,NaN
98,10000764,27897940,5,7802,9,10000764,M,86,2132,2014 - 2016,NaN


In [3]:
from utils_complications import who_get_this_disease, who_not_get_this_disease, batch_query_by_subject_id, query_by_subject_id

In [41]:
def construct_chi2_test(df_ibd, df_no_ibd, icd_code):
    a = len(who_get_this_disease(df_ibd, icd_code))
    b = len(who_not_get_this_disease(df_ibd, icd_code))
    c = len(who_get_this_disease(df_no_ibd, icd_code))
    d = len(who_not_get_this_disease(df_no_ibd, icd_code))
    
    table = np.array([[a, b], [c, d]]) 
    print(table)
    
    t1 = (a + c) * (a + b) / (a + b + c + d)
    t2 = (b + d) * (a + b) / (a + b + c + d)
    t3 = (a + c) * (b + d) / (a + b + c + d)
    t4 = (b + d) * (c + d) / (a + b + c + d)
    
    expected = np.array([[t1, t2], [t3, t4]]) 
    print(expected)
    
    _, pvalue = chisquare(table, expected, ddof=2)
    print(pvalue)

    return
    

In [46]:
construct_chi2_test(df_ibd, df_no_ibd, '2724')

[[   491   1926]
 [ 33448 147179]]
[[   448.14669151   1968.85330849]
 [ 27646.21946089 147136.14669151]]


ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
[0.2080358 0.       ]

In [43]:
def construct_fisher_test(df_ibd, df_no_ibd, icd_code):
    a = len(who_get_this_disease(df_ibd, icd_code))
    b = len(who_not_get_this_disease(df_ibd, icd_code))
    c = len(who_get_this_disease(df_no_ibd, icd_code))
    d = len(who_not_get_this_disease(df_no_ibd, icd_code))

    table = np.array([[a, b], [c, d]]) 
    print(table)
    oddsratio, pvalue = fisher_exact(table)
    print(pvalue)

    return
    

In [60]:
construct_fisher_test(df_ibd, df_no_ibd, 'V851')

[[    97   2320]
 [  1213 179414]]
1.0283264598967872e-41
